In [52]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
import os
import statistics
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [34]:
benign_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/benign/datas-final-by-origin-tool.pkl')

In [35]:
malware_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/malware/1/datas-by-origin-tool.pkl')
for i in [2,3,4,5,6,7,8,9,10,11,12,13,42,43,44,45,46,47,48,49,50,52,53,54]:
    pkl_path = '/home/sdsra/Downloads/CTU-13-Dataset/malware/{}/datas-by-origin-tool.pkl'.format(str(i))
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [36]:
path = '/home/sdsra/Downloads/CTU-13-Dataset/malware-2'
sub_dir_list = os.listdir(path)   
for dir in sub_dir_list:
    pkl_path = path +'/'+dir+'/datas-by-origin-tool.pkl'
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [37]:
malware_datas = malware_datas[malware_datas['label'] == 'botnet']

In [38]:
len(malware_datas)

72008

In [39]:
numeric_columns = list(filter(lambda x: x not in ['src_ip','src_port','dst_ip','dst_port','proto'], benign_datas.columns))

In [40]:
benign_datas['is_malware'] = 0
malware_datas['is_malware'] = 1

In [41]:
datas = pd.concat([benign_datas, malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [42]:
X_train, X_test, y_train, y_test = train_test_split(datas[numeric_columns], datas['is_malware'], test_size=0.2, random_state = 101)


In [43]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_columns])
X_test_scaled  = scaler.transform(X_test[numeric_columns]) 

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [44]:
kmeans = KMeans(n_clusters=2)  
kmeans.fit(X_train_scaled)  

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [45]:
y_pred = kmeans.predict(X_test_scaled)

In [46]:
confusion_matrix(y_test, y_pred)

array([[   16,  1330],
       [  823, 13589]])

In [53]:
TN = 16
FP = 1330
FN = 823
TP = 13589

In [54]:
accuracy = (TN + TP) / (TN + TP + FP + FN)

In [55]:
precision = TP / (TP + FN)

In [56]:
print('accuracy : {}, precision: {}'.format(accuracy, precision))

accuracy : 0.863370986165757, precision: 0.942894809880655
